# Práctica 1: Algoritmos genéticos
Hernández Jiménez Erick Yael. 2023630748
5BV1-Ingeniería en Inteligencia Artificial 2025-1

## Descripción del problema
> Fred and George Weasley are preparing themselves for a new term in Hogwarts School of Witchcraft and Wizardry. They are planning to sell the new products from their joke store Weasleys’ Wizard Wheezes to the students at the school. So, they are preparing a knapsack with as many products as possible to make a bigger proﬁt. Mrs. Weasley, their mother, has forbidden them to sell these products, so they must carry them hidden using a small knapsack. Currently, they have 10 products of each type.

## Modelado

### Cromosomas
Los cromosomas se codificarán como un arreglo de 7 elementos cuyo valor será un entero positivo entre 0 y 10 tal que:
- Decoy Denotators -> DD = 0
- Love Potion -> LP = 1
- Extendable Ears -> EE = 2
- Skiving Snackbox -> SS = 3
- Never Fudge -> NF = 4
- Puking Pastilles -> PP = 5
- Nosebleed Nougat -> NN = 6

|DD|LP|EE|SS|NV|PP|NN|
|-:|-:|-:|-:|-:|-:|-:|
|0 |1 |2 |3 |4 |5 |6 |

Para modelar los cromosomas usaremos los arreglos que la biblioteca de Numpy nos proporciona

In [7]:
import numpy as np

### Funciones de evaluación
#### Función de costo
Nuestra función de costo está dado por:
$$\sum_{i=0}^{6} (precio_{i})\cdot (alelo_i)$$
Donde:
- $i$: es el locus en el cromosoma
- $precio_i$: es el precio del objeto al que corresponde al gen $i$
- $alelo_i$: es el valor del gen $i$

En el código, se verá reflejado en un arreglo de precios tal que:
|DD|LP|EE|SS|NV|PP|NN|
|-:|-:|-:|-:|-:|-:|-:|
|10|8 |12|6 |3 |2 |2 |

In [2]:
precios = np.array([10,8,12,6,3,2,2])

#### Función de restricción
Mientras que la función de restricción está dada por la desigualdad:
$$0<\sum_{i=0}^{6} (peso_{i})\cdot (alelo_i) \leq 30$$
Donde:
- $i$: es el locus en el cromosoma
- $peso_i$: es el peso del objeto al que corresponde al gen $i$
- $alelo_i$: es el valor del gen $i$

En el código, se verá reflejado en un arreglo de pesos tal que:
|DD|LP|EE|SS|NV|PP|NN|
|-:|-:|-:|-:|-:|--:|-:|
|4 |2 |5 |5 |2 |1.5|1 |

In [3]:
costos = np.array([4,2,5,5,2,1.5,1])

### Restricciones
> In addition to the knapsack capacity restriction, the brothers have decided that there must be at least 3 Love Potions and 2 Skiving Snackbox, the former because is highly demanded and the later because includes an assortment of their products.

Para cumplir esto, desde la inicialización se generarán los cromosomas con números aleatorios en los alelos de 0 a 10 exceptuando los genes 1 y 3 cuyos dominios son de 3 a 10 y de 2 a 10 respectivamente. La función [`numpy.random.randint`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) asegura estas condiciones.

### Inicialización
Crearemos los 10 cromosomas iniciales con números aleatorios de 0 a 10:
```Python
cromosoma = np.random.randint(0, 11, size=7)
```
Luego alteramos el valor de la posición 1 correspondiente al objeto _Love Potion_:
```Python
cromosoma[1] = np.random.randint(3,11)
```
E igualmente modificamos la posición 3 correspondiente al objeto _Skiving Snackbox_:
```Python
cromosoma[3] = np.random.randint(2, 11)
```
Finalmente, para validar que los individuos son válidos para el problema, evaluaremos a los individuos con la función de restricción.
```Python
np.dot(cromosoma, costos)
```

### Población
Creamos la pobación de 10 individuos

In [4]:
poblacion = []

for i in range(10):
    cromosoma = np.zeros(7, dtype=int)
    while (np.dot(cromosoma, costos) <= 0) or (np.dot(cromosoma, costos) > 30):
        cromosoma = np.random.randint(0, 11, size=7)
        cromosoma[1] = np.random.randint(3,11)
        cromosoma[3] = np.random.randint(2, 11)
    poblacion.append(cromosoma)
    print(cromosoma, f" agregado con peso de {np.dot(cromosoma, costos)}\n")

print('-'*20)
for cromosoma in poblacion:
    print(cromosoma, " : ", np.dot(cromosoma, costos), "\n")

[0 4 0 2 1 0 8]  agregado con peso de 28.0

[1 4 0 3 0 0 3]  agregado con peso de 30.0

[0 4 0 2 1 5 1]  agregado con peso de 28.5

[0 6 0 2 1 1 0]  agregado con peso de 25.5

[0 3 0 2 4 2 0]  agregado con peso de 27.0

[1 3 0 3 1 0 0]  agregado con peso de 27.0

[0 5 0 3 0 0 1]  agregado con peso de 26.0

[0 4 0 3 0 2 0]  agregado con peso de 26.0

[1 3 1 2 1 0 1]  agregado con peso de 28.0

[0 3 1 2 2 1 2]  agregado con peso de 28.5

--------------------
[0 4 0 2 1 0 8]  :  28.0 

[1 4 0 3 0 0 3]  :  30.0 

[0 4 0 2 1 5 1]  :  28.5 

[0 6 0 2 1 1 0]  :  25.5 

[0 3 0 2 4 2 0]  :  27.0 

[1 3 0 3 1 0 0]  :  27.0 

[0 5 0 3 0 0 1]  :  26.0 

[0 4 0 3 0 2 0]  :  26.0 

[1 3 1 2 1 0 1]  :  28.0 

[0 3 1 2 2 1 2]  :  28.5 



### Padres
Una vez que hemos creado la población, seleccionaremos a los padres.

#### Selección por ruleta
Primero calcularemos la aptitud total de los individuos en la población tal que
$$\sum_{i=1}^{10} (\sum_{j=0}^{6} (precio_{j})\cdot (alelo_i))_i$$

In [5]:
aptitud_total:int = 0
aptitudes_individuales = np.zeros(len(poblacion), dtype=int)
for i in range(len(poblacion)):
    aptitudes_individuales[i] = np.dot(poblacion[i], precios)
    print(f"La aptitud del individuo '{poblacion[i]}' es de {aptitudes_individuales[i]}")
    aptitud_total += np.dot(poblacion[i], precios)
print('-'*20, f"\n La aptitud total es de {aptitud_total}\n")


La aptitud del individuo '[0 4 0 2 1 0 8]' es de 63
La aptitud del individuo '[1 4 0 3 0 0 3]' es de 66
La aptitud del individuo '[0 4 0 2 1 5 1]' es de 59
La aptitud del individuo '[0 6 0 2 1 1 0]' es de 65
La aptitud del individuo '[0 3 0 2 4 2 0]' es de 52
La aptitud del individuo '[1 3 0 3 1 0 0]' es de 55
La aptitud del individuo '[0 5 0 3 0 0 1]' es de 60
La aptitud del individuo '[0 4 0 3 0 2 0]' es de 54
La aptitud del individuo '[1 3 1 2 1 0 1]' es de 63
La aptitud del individuo '[0 3 1 2 2 1 2]' es de 60
-------------------- 
 La aptitud total es de 597



Una vez calculada la aptitud total e individual, calcularemos la probabilidad acumulada sumando las probabilidades de selección de cada individuo:
$$P_{set} = \frac{Aptitud_i}{Aptitud\_Total}$$

In [6]:
probabilidad_acumulada = np.zeros(len(poblacion), dtype=float)

for i in range(len(poblacion)):
    probabilidad_individual: float = aptitudes_individuales[i]/aptitud_total
    probabilidad_acumulada[i] = probabilidad_acumulada[i-1] + probabilidad_individual
print("Las probabilidades son:\n")
for i in range(len(poblacion)):
    print(f"Individuo en la posición {i}: {probabilidad_acumulada[i]}\n")

Las probabilidades son:

Individuo en la posición 0: 0.10552763819095477

Individuo en la posición 1: 0.21608040201005024

Individuo en la posición 2: 0.3149078726968174

Individuo en la posición 3: 0.423785594639866

Individuo en la posición 4: 0.5108877721943048

Individuo en la posición 5: 0.6030150753768844

Individuo en la posición 6: 0.7035175879396984

Individuo en la posición 7: 0.793969849246231

Individuo en la posición 8: 0.8994974874371858

Individuo en la posición 9: 0.9999999999999999



Finalmente, se seleccionan los padres a cruzar con dos número uniformemente aleatorio entre 0 y 1. Así, se seleccionan aquellos individuos a los que les corresponda los números asignados en su rango dentro de la probabilidad acumulada. Para ello se usa la función [`numpy.random.rand`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html) que asegura estas características.
Además de esto, en la selección del segundo padre, debemos asegurar que se seleccionen un individuo distinto ya que no se pueden reproducir a sí mismos.

In [162]:
padres_seleccionados = []
# Seleccionando al primer padre
individuo_1: int = 0
frecuencia: float = np.random.rand()
print(f"Se selecciona la frecuencia {frecuencia}\n")
individuo_1 = np.searchsorted(probabilidad_acumulada, frecuencia)
individuo_1 = min(individuo_1, len(poblacion) - 1)
padres_seleccionados.append(poblacion[individuo_1])
print(f"Se ha agregado al individuo en la posición {individuo_1}: {padres_seleccionados[0]}\n")

# Seleccionando al segundo padre
individuo_2: int = individuo_1
while individuo_2 == individuo_1:
    frecuencia: float = np.random.rand()
    print(f"Se selecciona la frecuencia {frecuencia}\n")
    individuo_2 = np.searchsorted(probabilidad_acumulada, frecuencia)
    individuo_2 = min(individuo_2, len(poblacion) - 1)
padres_seleccionados.append(poblacion[individuo_2])
print(f"Se ha agregado al individuo en la posición {individuo_2}: {padres_seleccionados[1]}\n")

Se selecciona la frecuencia 0.1558652735964816

Se ha agregado al individuo en la posición 1: [0 6 0 2 0 0 5]

Se selecciona la frecuencia 0.9528101046785575

Se ha agregado al individuo en la posición 9: [1 7 0 2 1 0 0]



#### Probabilidad de cruza
Para que los individuos se crucen, primero se debe decidir su cruza a partir de la probabilidad de cruza. En esta práctica se tratará con una probabilidad $P_c = 0.85$. Si los padres se cruzan, en este programa se mantendrán en el mismo arreglo para posteriormente cruzarlos y hacerlos competir con sus hijos, de lo contrario, se pasarán como supervivientes para evaluar si tendrán mutación o no.

In [ ]:
#hijos = []
#probabilidad_cruza = 0.85
#random:float = np.random.rand()
#print(f"La probabilidad resultante es de {random}")
#if random > probabilidad_cruza:
#    for padres in padres_seleccionados:

### Hijos
Para generar a los hijos, se han dado los siguientes parámetros:
|Característica|Valor y descripción|
|-:|-:|
|Probabilidad de cruza| 0.85|
|Cruza|Cruza uniforme|
|Umbral|$u\leq 0.5$|

#### Cruza uniforme
Esta cruza se caracteriza por tratar los genes de manera independiente y elige aleatoriamente a los padres de los que cada hijo heredará cada gen.